In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import time
import sklearn.metrics as mt
import hyperopt
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

#Additional packages for the clustering part
from sklearn.decomposition import PCA
from sklearn import cluster
from sklearn.metrics import silhouette_score

# Task 1a) 
Load data from *Labelled_Multiclass.csv* in pandas dataframe, delete unnecessary info (idlink, eqtype, acmLowerMode, freqband, bandwidth), and visualize it in tabular form 

Hint:
- *Pandas* guide at: https://pandas.pydata.org/docs/user_guide/index.html

In [ ]:
raw_data = pd.read_csv('Labelled_Multiclass.csv')

# F: use another pandas dataframe "data" to store the new dataframe where you have dropped unnecessary columns

############# ADD YOUR CODE BELOW #############

columns = raw_data.columns.to_numpy()

# delete the first 8 columns
data = raw_data.drop(columns=columns[:8])

#print(data.iloc[2,:])
#print(data.to_numpy()[2,:]) #F: same as print(data.iloc[2,:].to_numpy())
data


# Task 1b) 
Define function *plot_feature()* that takes in input dataframe and feature number (passed as integer representing the column index), and plots the distribution of the feature. Distribution should be plotted as histograms and boxplot

(code is already given below)

In [ ]:
def plot_feature(dataframe,featurenumber=0):
    feature = dataframe.iloc[:,featurenumber].to_numpy() #F: this way it is a numpy.ndarray
    print('You are going to see feature {}: {}'.format(featurenumber,dataframe.columns.values[featurenumber]))
    minvalue = feature.min()
    maxvalue = feature.max()
    
    plt.hist(feature, bins = 1 + int(maxvalue-minvalue))
    plt.xlabel(dataframe.columns.values[featurenumber])
    plt.ylabel('No. of occurrences')
    fig_hist = plt.gcf()
    plt.show()
    
    allbox = []
    
    #F: boxplot of the feature passed as input param
    dataframe.iloc[:,featurenumber].plot(kind='box', subplots=True, sharex=False, sharey=False)
    fig_box = plt.gcf()
    plt.show()
    
    #F: boxplot of all the features in the same graph
    #dataframe.iloc[:,0:len(dataframe.columns)].plot(kind='box', subplots=False, sharex=False, sharey=False)
    #plt.xticks(rotation=90)
    #fig_allbox = plt.gcf()
    #plt.show()
    
    
    #F: in case you want to save figures in a dedicated folder
    #res_folder = 'Figs_features'
    #if not os.path.exists(res_folder):
    #    os.makedirs(res_folder)
    #
    #fig_hist.savefig(res_folder + '\Feature_{}_hist.png'.format(dataframe.columns.values[featurenumber]))
    #fig_box.savefig(res_folder + '\Feature_{}_boxplot.png'.format(dataframe.columns.values[featurenumber]))
    ##fig_allbox.savefig(res_folder + '\All-features_boxplot.png')
    

# Task 1c)
Call function *plot_feature()* to plot features no. 2 and 4

(code is already given below)

In [ ]:
plot_feature(data,2)

plot_feature(data,4)


### What can we observe? 

# Task 1d) 
Check which features have missing values. Then, ONLY FOR THOSE FEATURES, use function *plot_feature()* from task 1b) to plot the distribution neglecting missing values

If everything is ok in your code, you should find 24 features with NaN values. 
### Anything in common between these 24 features? 

Hints: 
- To plot each feature, you can use a support temporary dataframe extracting one column at a time from the original loaded dataframe
- Use *isnull* and *dropna* from *pandas* to find and drop NaN:

https://pandas.pydata.org/docs/reference/api/pandas.isnull.html

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html

In [ ]:
columns = raw_data.columns.to_numpy()
data = raw_data.drop(columns=columns[:8]) #F: this line (repeated from a previous cell) re-loads data with nan, thus prevents the case of not being able to find Nans (in case you run this cell multiple times)
featureswithnan = 0 #F: this is supposed to count the number of features with nan
############# ADD YOUR CODE BELOW #############


for i in range(len(data.columns.values)):
    tempdata=data.iloc[:,i] #F: tempdata is now a pandas.Series 
    tempdata=tempdata.to_frame() #F: need to convert pandas.Series into dataframe to pass it into function plot_feature() above, due to indexing error in that function otherwise
    # ---------------------------------------------------------
    # Find indexes of cells with Nan
    nanindexes=np.where(pd.isnull(tempdata))
    #print(nanindexes)
    #print(len(np.where(pd.isnull(raw_data))[0])) #rows
    #print(len(np.where(pd.isnull(raw_data))[1])) #columns
    
    # ---------------------------------------------------------
    # Find indexes of empty cells 
    #emptyindexes=np.where(data.applymap(lambda x: x == ''))
    
    if len(nanindexes[0])!=0:
        featureswithnan += 1
        print('Feature {} has Nan. We are going to remove Nan and plot the feature distribution.'.format(tempdata.columns.values[0]))
        tempdata.dropna(inplace=True)
        #tempdata.drop(tempdata[tempdata[tempdata.columns.values[0]]=='18'].index, inplace=True) #F: use this if you want to drop a row with a specific value (e.g., =18) 
        plot_feature(tempdata,)

#################################################
print('*********************************')
print('We found {} features with NaN values'.format(featureswithnan))        
print('*********************************')

# Task 1e) 
Take features with missing values and substitute NaN with the median of the feature without NaN. Print the features and corresponding median values used to replace NaN values

Hint: 
- *median(skipna=True)* from *Pandas* calculates median on dataframes neglecting Nan values

In [ ]:
data = raw_data.drop(columns=columns[:8]) #F: this line re-loads data with nan, thus prevents errors (not being able to find Nans) in case you run this cell multiple times

nanindexes=np.where(pd.isnull(data)) # store indexes where data has nan values

for i in np.unique(nanindexes[1]): #F: scan all columns where we have nan 
    # np.unique finds the unique (i.e., wothout repetitions) elements of an array 
    # "[1]" so we focus on the columns (i.e., the features)
    
    # For each feature i you should replace its Nan values with median of the valid values of the feature
    ############# ADD YOUR CODE BELOW #############

    tempdata = data.iloc[:,i] #F: extracts the i-th column from data, where i is one of the column with nan
    median = tempdata.median(skipna=True) #F: calculate median for this column neglecting Nan
    print('{}, median ={}'.format(data.columns.values[i],median))
    data[data.columns.values[i]] = data[data.columns.values[i]].fillna(median) #F: insert the median in the dataset

    
#############################################################################
#Following lines verify that nan in data have been replaced
#if nanindex turns out to be an array with empty elements --> OK
#if necessary, change "data" with the name of your dataframe to make it work
nanindexes = np.where(pd.isnull(data))
nanindexes

# Task 2a) 
Define function *load_dataset()* to load a dataset from a given dataframe in input into given arrays X and y passed in input

In [ ]:
def load_dataset(X, y, dataset):
#Inputs: - X: (numpy array) features retrieved from dataset
#        - y: (numpy array) labels retrieved from dataset
#        - dataset: dataframe with data points to be read (it must be a dataset without unnecessary features as created above)
#This function should load into X and y in input the datapoints retrieved from dataset and return X and y
############# ADD YOUR CODE BELOW #############

    X = dataset.iloc[:,0:-1].to_numpy()
    y = dataset.iloc[:,-1].to_numpy()
    
    return X, y


# Task 2b)
Test function *load_dataset()* with datasets created in task 1e)

(code is already given below)

In [ ]:
# you should pass empty X and y to function load_window_dataset
X=None 
y=None

X, y = load_dataset(X, y, data)

print(X)
print(y)
print(X.shape)
print(y.shape)


# Task 3a) 
Standardize features, split dataset into train/test (80% / 20%) with balanced classes, and print shapes of train and test sets

Hints: 
- use *StandardScaler()* from *sklearn.preprocessing* and *train_test_split* from *sklearn.model_selection* 

(code is already given below)

In [ ]:
test_percentage = 0.2  # Percentage of points included in the test dataset (e.g., 20% = 0.2)

scaler = StandardScaler()
X = scaler.fit_transform(X)

# -------------------- Scale the data (Manually) --------------------
# mean_X = np.mean(X, axis=0)  # Mean per feature of the training data
# std_tX = np.std(X, axis=0)  # Std per feature of the training data
# X = (X - mean_X) / std_X  # Scaling training data
# -------------------------------------------------------------------

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_percentage,
                                                    random_state=42, stratify=y)

#print(X_train)
#print(X_test)
#print(y_train)
#print(y_test)
print('Training set shape (features): {}'.format(X_train.shape))
print('Training set shape (labels): {}'.format(y_train.shape))
print('Test set shape (features): {}'.format(X_test.shape))
print('Test set shape (labels): {}'.format(y_test.shape))



# Task 3b)
Define function *optimize_RF()* that performs hyperparameter optimization with 5-fold crossvalidation, retrains the model on the entire training set with the optimized hyperparameters and returns the trained model (details below).

Hints: 
- use *hyperopt* library to perform hyperparameters optimization: http://hyperopt.github.io/hyperopt/
- use *RandomForestClassifier from sklearn.ensemble* for the RF model: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
- use *cross_val_score* as a metric to determine the best hyperparameters: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

(code is already given below)

In [ ]:
def optimize_RF(X_train, y_train): 
#Inputs: - X_train: training set (features)
#        - y_train: training set (ground truth labels)
#
#Output: - rf: model to be returned by the function
#
#This function should: 
#         * Perform RF hyperparameters optimization via crossvalidation
#         * Print best hyperparameters obtained with crossvalidation
#         * Print best crossvalidation accuracy (across all hyperparameters combinations) and duration
#         * Retrain a new RF model with best hyperparameters using the entire training set (X_train, y_train)
#         * Print training results (best accuracy and training duration)
#         * Return the trained RF model

    #F: define the search space for your hyperparameters
    space4rf = { 
     'estimators': hp.choice('estimators', [10, 50, 100, 200, 500]),
     'crit': hp.choice('crit', ['gini', 'entropy', 'log_loss']),
     'maxd': hp.choice('maxd', np.arange(1, 20, 2)),
    }

    def hyperopt_train_test(params):
        model = RandomForestClassifier(n_estimators=params['estimators'], criterion=params['crit'],
                                       max_depth=params['maxd'], class_weight='balanced')

        return cross_val_score(model, X_train, y_train, cv = 5).mean()
        #F: cross_val_score is from scikit learn (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)
        #F: will use the default score (for RF it is accuracy)
        #F: this includes also training; cv=5 (5-folds crossvalidation)
        #F: .mean() is taken as cross_val_score returns an array of scores (one for each fold)
        
    def f(params): #F: this function is used below, as a parameter to fmin
        acc = hyperopt_train_test(params)
        return {'loss': -acc, 'status': STATUS_OK} #F: loss is returned as opposite (negative) of accuracy because we will use in f_min (that only minimizes), where we want to minimize the loss (i.e., maximize accuracy)

    trials = Trials() #F: an object that keeps track of all trials (i.e., combination of hyperparameters) tested during the optimization
    
    ta = time.time()
    best_params = fmin(f, space4rf, algo=tpe.suggest, max_evals=10, trials=trials)
    #F: see: https://github.com/hyperopt/hyperopt/blob/master/hyperopt/fmin.py
    #F: at this point, best_param is a dictionary where each key is the index of the corresponding best param in space4rf
    tb = time.time()
    print(best_params)
    
    best_params = hyperopt.space_eval(space4rf, best_params)
    #F: this is used to extract from space4rf the best values 
    #   according to the indexes in best_params (and put such values in best_params)
    print(best_params)
    
    best_cv_acc = -round(trials.best_trial['result']['loss'], 2) #F: best across trials
    print('best_cv_acc: ' + str(best_cv_acc))
    print('Crossvalidation duration for RF is {} s\n'.format(round(tb - ta)))

    
    #######################################################################################################
    #F: Now you have best hyperparameters obtained with crossvalidation and should train a new model 
    #   using those hyperparameters and the entire training set; then return the trained model
    
    rf = RandomForestClassifier(n_estimators=best_params['estimators'], criterion=best_params['crit'],
                                       max_depth=best_params['maxd'], class_weight='balanced')
    
    t0 = time.time()
    rf.fit(X_train, y_train)
    t1 = time.time()

    print('Best number of estimators: {}\n'.format(best_params['estimators']))
    print('Best splitting criterion: {}\n'.format(best_params['crit']))
    print('Best max_depth: {}\n'.format(best_params['maxd']))
    print('Crossvalidation accuracy: {}\n'.format(best_cv_acc))
    print('Training duration for RF is {} s\n'.format(round(t1 - t0)))


    return rf


# Task 3c)
Call function *optimize_RF()* and save the trained RF model in a .json file

Hint:
- Use pickle.dump to save models to disk

(code is already given below)

In [ ]:
print('Training RF...')
rf = optimize_RF(X_train, y_train)

if not os.path.exists('models'):
    os.makedirs('models')
        
rfmodelfile = 'models\RF.json'
pickle.dump(rf, open(rfmodelfile, 'wb'))


# Task 3d)
Define function *optimize_KNN()* that performs hyperparameter optimization with 5-fold crossvalidation, retrains the model on the entire training set with the optimized hyperparameters and returns the trained model (details below).

Hints: 
- take inspiration from task 3b) on RF optiomization using *hyperopt* library
- use *KNeighborsClassifier from sklearn.neighbors* for the KNN model: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [ ]:
def optimize_KNN(X_train, y_train): 
#Inputs: - X_train: training set (features)
#        - y_train: training set (ground truth labels)
#
#Output: - knn: model to be returned by the function
#
#This function should: 
#         * Perform KNN hyperparameters optimization via crossvalidation
#         * Print best hyperparameters obtained with crossvalidation
#         * Print best crossvalidation accuracy (across all hyperparameters combinations) and duration
#         * Retrain a new KNN model with best hyperparameters using the entire training set (X_train, y_train)
#         * Print training results (best accuracy and training duration)
#         * Return the trained KNN model

    #F: define the search space for your hyperparameters
    space4knn = { 
     'neighb': hp.choice('neighb', [1, 5, 10, 20, 50, 100]),
     'wgts': hp.choice('wgts', ['uniform', 'distance']),
    }
    ############# ADD YOUR CODE BELOW #############


    def hyperopt_train_test(params):
        model = KNeighborsClassifier(n_neighbors=params['neighb'], weights=params['wgts'])

        return cross_val_score(model, X_train, y_train, cv = 5).mean()
        #F: cross_val_score is from scikit learn (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)
        #F: will use the default score (for KNN it is accuracy)
        #F: this includes also training; cv=5 (5-folds crossvalidation)
        #F: .mean() is taken as cross_val_score returns an array of scores (one for each fold)
        
    def f(params): #F: this function is used below, as a parameter to fmin
        acc = hyperopt_train_test(params)
        return {'loss': -acc, 'status': STATUS_OK} #F: loss is returned as opposite (negative) of accuracy because we will use in f_min (that only minimizes), where we want to minimize the loss (i.e., maximize accuracy)

    trials = Trials()
    ta = time.time()
    best_params = fmin(f, space4knn, algo=tpe.suggest, max_evals=10, trials=trials)
    tb = time.time()
    print(best_params)
    
    best_params = hyperopt.space_eval(space4knn, best_params)
    #F: this is used to extract from space4knn the best values according to the indexes in best_params (and put such values in best_params)
    print(best_params)
    
    best_cv_acc = -round(trials.best_trial['result']['loss'], 2) #F: best across trials
    print('best_cv_acc: ' + str(best_cv_acc))
    print('Crossvalidation duration for KNN is {} s\n'.format(round(tb - ta)))

    #######################################################################################################
    #F: Now you have best hyperparameters obtained with crossvalidation and should train a new model 
    #   using the entire training set using those hyperparameters and return the trained model
    ############# ADD YOUR CODE BELOW #############
    
    knn = KNeighborsClassifier(n_neighbors=best_params['neighb'], weights=best_params['wgts'])
    
    t0 = time.time()
    knn.fit(X_train, y_train)
    t1 = time.time()

    print('Best number of neighbors: {}\n'.format(best_params['neighb']))
    print('Best weight function: {}\n'.format(best_params['wgts']))
    print('Crossvalidation accuracy: {}\n'.format(best_cv_acc))
    print('Training duration for KNN is {} s\n'.format(round(t1 - t0)))


    return knn



# Task 3e)
Call function *optimize_KNN()* and save the trained KNN model in a .json file

Hint:
- See task 3c) for reference

(code is already given below)

In [ ]:
print('Training KNN...')
knn = optimize_KNN(X_train, y_train)

if not os.path.exists('models'):
    os.makedirs('models')
        
knnmodelfile = 'models\KNN.json'
pickle.dump(knn, open(knnmodelfile, 'wb'))

# Task 4a)
Define function *performance_eval()* that takes in input ground truth and predicted labels, prints results in output, and returns PER-CLASS metrics (details below)
**N.B. Metrics should be calculated "manually" (applying the specific formulae) AND with sklearn APIs**

Hint: 
- *confusion_matrix* and other metrics from *sklearn.metrics* can be used to compute metrics via APIs
- parameter *average=None* is used to calculate unweighted metrics using sklearn APIs
- you can use pandas dataframes to calculate metrics "manually"



In [ ]:
def performance_eval(y_true, y_pred, l_names):
# Inputs: - y_true: ground truth labels
#         - y_pred: predicted labels
#         - l_names: labels used in the problem at hand (e.g., 0,1,2,3,4,5)
# Outputs: - return accuracy, precision, recall, f1score
# This function should: 
#         - Compute Accuracy, Precision (per class), Recall (per class), F1-score (per class). 
#         - Metrics should be calculated "manually" (applying the specific formulae) AND with sklearn APIs
#         - Compute confusion matrix (cm)
#         - Print metrics and cm in output (already given below)
#         - Return Accuracy, Precision, Recall, F1-score (note that the last 3 metrics are per-class) (already given below)
#         - Draw the confusion matrix (already given in the code, you need to call the confusion matrix object as cm)
    vectors = pd.DataFrame()
    vectors['ground_truth'] = y_test
    vectors['predicted'] = y_pred
    
    num_labels = len(set(l_names))
    
    # Metrics calculated with sklearn APIs
    accuracy = mt.accuracy_score(y_true, y_pred)
    precision = mt.precision_score(y_true, y_pred, labels=l_names, average=None) #F: average=None gives per-class results
    recall = mt.recall_score(y_true, y_pred, labels=l_names, average=None)
    f1score = mt.f1_score(y_true, y_pred, labels=l_names, average=None)
    cm = confusion_matrix(y_true, y_pred)
    
    #F: init
    man_accuracy = 0
    man_precision = np.zeros(num_labels)
    man_recall = np.zeros(num_labels)
    man_f1score = np.zeros(num_labels)
    
    # Now you should compute "manually" (applying the specific formulae)
    ############# ADD YOUR CODE BELOW #############
    
    # ------------ accuracy ------------
    man_accuracy = len(vectors.loc[vectors['ground_truth'] == vectors['predicted']]) / len(vectors)
    
    # ------------ precision ------------
    for i in range(num_labels):
        measure = pd.DataFrame()
        measure['ground_truth'] = vectors.loc[vectors['predicted'] == i]['ground_truth']
        measure['predicted'] = vectors.loc[vectors['predicted'] == i]['predicted']
        man_precision[i] = len(measure.loc[measure['ground_truth'] == measure['predicted']]) / len(measure)

    # ------------ recall ------------
    for i in range(num_labels):
        measure = pd.DataFrame()
        measure['ground_truth'] = vectors.loc[vectors['ground_truth'] == i]['ground_truth']
        measure['predicted'] = vectors.loc[vectors['ground_truth'] == i]['predicted']
        man_recall[i] = len(measure.loc[measure['ground_truth'] == measure['predicted']]) / len(measure)

    # ------------ F1-score ------------
    for i in range(num_labels):
        man_f1score[i] = 2 * (man_precision[i] * man_recall[i]) / (man_precision[i] + man_recall[i])

        
    ########################################################################
    print('Accuracy: {}'.format(accuracy))
    print('Manual accuracy: {}'.format(man_accuracy))

    print('Precision per class: {}'.format(precision))
    print('Manual precision per class: {}'.format(man_precision))

    print('Recall per class: {}'.format(recall))
    print('Manual recall per class: {}'.format(man_recall))

    print('F1-score per class: {}'.format(f1score))
    print('Manual F1-score per class: {}'.format(man_f1score))

    print('confusion matrix:\n {}'.format(cm))
    

    #F: in the following, we plot the confusion matrix (cm)
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=l_names, yticklabels=l_names,
           title= 'Confusion matrix',
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # Loop over data dimensions (#F: i.e., cells in the confusion matrix) and create text annotations. 
    fmt = 'd'
    thresh = cm.max() / 2.
    for w in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, w, format(cm[w, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[w, j] > thresh else "black")
    fig.tight_layout()
    plt.show()
        
    return accuracy, precision, recall, f1score

# Task 4b) 
Load RF and KNN models saved in .json files in tasks 3c) and 3e) and store into new model objects.
Then, perform predictions for the test set and call function *performance_eval()* to evaluate performance of the RF and KNN models

Hints:
- use *pickle.load* to load .json models

(code is already given below)

In [ ]:
# list of all the distinct labels
label_names = list(set(y))
print(label_names)

#F: the following objects are those where you need to load the pre-saved RF and KNN models
newRF = RandomForestClassifier()
newKNN = KNeighborsClassifier()

print('Evaluating RF performance.....')
newRF = pickle.load(open(rfmodelfile, 'rb'))
ypredRF = newRF.predict(X_test)
accuracy_RF, precision_RF, recall_RF, f1_RF = performance_eval(y_test, ypredRF, label_names)

print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('Evaluating KNN performance.....')
newKNN = pickle.load(open(knnmodelfile, 'rb'))
ypredKNN = newKNN.predict(X_test)
accuracy_KNN, precision_KNN, recall_KNN, f1_KNN = performance_eval(y_test, ypredKNN, label_names)


# Task 4c) 
For RF and KNN, compute global precision, recall and F1-score as mean of per-class metrics and plot in 4 separate graphs: 
1) global metrics (accuracy, precision, recall, F1-score) - bar graph
2) precision (global and per-class) - bar-graph + global precision in a horizontal line
3) recall (global and per-class) - bar-graph + global recall in a horizontal line
4) F1-score (global and per-class) - bar-graph + global F1-score in a horizontal line

(code is already given below)

In [ ]:
#---------------RF---------------#
global_precision_RF = precision_RF.mean()
global_recall_RF = recall_RF.mean()
global_f1_RF = f1_RF.mean()

RF_global = [accuracy_RF, global_precision_RF, global_recall_RF, global_f1_RF]

#---------------KNN---------------#
global_precision_KNN = precision_KNN.mean()
global_recall_KNN = recall_KNN.mean()
global_f1_KNN = f1_KNN.mean()

KNN_global = [accuracy_KNN, global_precision_KNN, global_recall_KNN, global_f1_KNN]

#---------------plots---------------#
xRF = np.arange(4)-0.1  # the x locations for the bars in global metrics plot
xKNN = np.arange(4)+0.1  # the x locations for the bars in global metrics plot
x2RF = np.arange(6)-0.1  # the x locations for the bars in precision/recall/f1-score metrics plot
x2KNN = np.arange(6)+0.1  # the x locations for the bars in precision/recall/f1-score metrics plot
w = 0.2       # the width of the bars

# 1) global metrics
plt.bar(xRF, RF_global, width=w, edgecolor='black', color='c', align='center', hatch='///', label='RF')
plt.bar(xKNN, KNN_global, width=w, edgecolor='black', color='y', align='center',hatch='---', label='KNN')
plt.xticks(xRF+0.1, ["Accuracy", "Precision", "Recall", "F1-Score"])
plt.title('RF & KNN performance metrics')
plt.ylabel('Score')
plt.ylim([0.5,1.01])
plt.grid()
plt.legend()
plt.show()

# 2) precision (global and per-class)
plt.bar(x2RF, precision_RF, width=w, edgecolor='black', color='c', align='center', hatch='///', label='RF')
plt.bar(x2KNN, precision_KNN, width=w, edgecolor='black', color='y', align='center', hatch='---', label='KNN')
plt.plot(x2RF,list((global_precision_RF,)*6), color='c', linestyle='dashed')
plt.plot(x2RF,list((global_precision_KNN,)*6), color='y', linestyle='dashed')
plt.xticks(x2RF+0.1, label_names)
plt.title('RF & KNN precision (global and per-class)')
plt.xlabel('Class')
plt.ylabel('Precision')
plt.ylim([0.5,1.01])
plt.legend()
plt.grid()
plt.show()

# 3) recall (global and per-class)
plt.bar(x2RF, recall_RF, width=w, edgecolor='black', color='c', align='center', hatch='///', label='RF')
plt.bar(x2KNN, recall_KNN, width=w, edgecolor='black', color='y', align='center', hatch='---', label='KNN')
plt.plot(x2RF,list((global_recall_RF,)*6), color='c', linestyle='dashed')
plt.plot(x2RF,list((global_recall_KNN,)*6), color='y', linestyle='dashed')
plt.xticks(x2RF+0.1, label_names)
plt.title('RF & KNN recall (global and per-class)')
plt.xlabel('Class')
plt.ylabel('Recall')
plt.ylim([0.5,1.01])
plt.legend()
plt.grid()
plt.show()

# 4) f1-score (global and per-class)
plt.bar(x2RF, f1_RF, width=w, edgecolor='black', color='c', align='center', hatch='///', label='RF')
plt.bar(x2KNN, f1_KNN, width=w, edgecolor='black', color='y', align='center', hatch='---', label='KNN')
plt.plot(x2RF,list((global_f1_RF,)*6), color='c', linestyle='dashed')
plt.plot(x2RF,list((global_f1_KNN,)*6), color='y', linestyle='dashed')
plt.xticks(x2RF+0.1, label_names)
plt.title('RF & KNN F1-score (global and per-class)')
plt.xlabel('Class')
plt.ylabel('F1-score')
plt.ylim([0.5,1.01])
plt.legend()
plt.grid()
plt.show()


#  CLUSTERING (UNSUPERVISED) PART STARTS HERE

In [ ]:
#F: check dimensions of current datasets (full, train, test)
print(X.shape)
print(y.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


# Task 5a) 
Starting from (X,y), above, generate new "truncated" dataset by removing all labels different from HW failures (label = 5)

(code is already given below)

In [ ]:
labels_to_keep = [5]
label_names = list(set(y)) #we had already used this line

#F: initialization
X_truncated = X
y_truncated = y


#F: now we remove all data points with label != labels_to_keep

for el in label_names:
    if el not in labels_to_keep:
        X_truncated = np.delete(X_truncated,np.where(y_truncated==el),0)
        y_truncated = np.delete(y_truncated,np.where(y_truncated==el),0)

print(X_truncated.shape)
print(y_truncated.shape)

# Task 5b) 
Instantiate and fit a PCA object on the truncated dataset *X_truncated*, then plot the explained variance for all components separately and cumulatively (i.e., adding one component at a time in the dataset) 

Hints: 
- use *PCA* and *PCA.fit* from *sklearn.decomposition* to instantiate and fit PCA object
- use *explained_variance_ratio_* to retrieve explained variance

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

(code is already given below)

In [ ]:
pca = PCA(n_components=35)
pca.fit(X_truncated)
print('Variance represented by each component, %:')
print(np.round(pca.explained_variance_ratio_, 3) * 100)

components = len(pca.explained_variance_ratio_)
plt.bar(range(1,components+1), pca.explained_variance_ratio_ * 100)
plt.xlabel('Number of components')
plt.ylabel('Per-component explained variance (%)')
plt.grid()
plt.show()

plt.plot(range(1,components+1), np.cumsum(pca.explained_variance_ratio_ * 100))
plt.xlabel('Number of components included')
plt.ylabel('Cumulative explained variance (%)')
plt.grid()
plt.show()


# Task 5c) 
Consider the cases of 2 and 3 PCA components to visualize the (transformed) dataset via a scatterplot of the (transformed) features in 2D and 3D graphs

Hints: 
- use *PCA.fit_transform* from *sklearn.decomposition* to transform the dataset into a 2D or 3D features space

(code is already given below)

In [ ]:
for i in [2, 3]:
    
    pca = PCA(i)
    X_PCA = pca.fit_transform(X_truncated)
        
    fig = plt.figure(figsize=(16,10))
            
    if i == 2:
        plt.scatter(X_PCA[:,0], X_PCA[:,1])
                
    elif i == 3:
        ax = fig.add_subplot(projection='3d')
        ax.scatter(X_PCA[:,0], X_PCA[:,1], X_PCA[:,2])
                
    plt.title('{}'.format(i))
    plt.show()

# Task 5d) 
Perform clustering on dataset X_truncated with number of clusters k in range(2, 21) and considering kmeans algorithm.
Plot inertia and silhouette for each value of k

Hints: 
- use *cluster* from *sklearn* to instantiate *KMeans* object and *inertia_* attribute to retrieve inertia from it
- use *silhouette_score()* and *labels_* attribute to retrieve silhouette

(code is already given below)

In [ ]:
inertia = {}
silhouette = {}
for k in range(2, 21):
    kmeans = cluster.KMeans(n_clusters=k, max_iter=1000).fit(X_truncated)
    inertia[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their cluster center
    silhouette[k] = silhouette_score(X_truncated, kmeans.labels_) 
plt.figure()
plt.plot(list(inertia.keys()), list(inertia.values()))
plt.xlabel("Number of clusters")
plt.ylabel("Inertia")
plt.show()

plt.figure()
plt.plot(list(silhouette.keys()), list(silhouette.values()))
plt.xlabel("Number of clusters")
plt.ylabel("Silhouette")
plt.show()

# Task 5e) 
Repeat task 5d) considering the PCA-modified dataset with a number of components that retains at least 90% variance of the original dataset X_truncated

Hint: 
- check output of task 5b) to identify the number of components to use
- take inspiration from task 5c) to apply PCA

In [ ]:
#F: for each number of cluster (k in k-means algorithm) store inertia and silhouette in the following dictionaries
inertia = {}
silhouette = {}
############# ADD YOUR CODE BELOW #############

numcomponents = 10
pca = PCA(numcomponents)
X_PCA = pca.fit_transform(X_truncated)

for k in range(2, 21):
    km = cluster.KMeans(n_clusters=k, max_iter=1000).fit(X_PCA)
    inertia[k] = km.inertia_ # Inertia: Sum of distances of samples to their cluster center
    silhouette[k] = silhouette_score(X_PCA, km.labels_) 
    

########################################################################
plt.figure()
plt.plot(list(inertia.keys()), list(inertia.values()), color='r')
plt.xlabel("Number of clusters")
plt.ylabel("Inertia (with {} components)".format(numcomponents))
plt.show()

plt.figure()
plt.plot(list(silhouette.keys()), list(silhouette.values()), color='r')
plt.xlabel("Number of clusters")
plt.ylabel("Silhouette (with {} components)".format(numcomponents))
plt.show()


# Task 6a)
Generate a new "truncated" dataset taking only labels 1=Extra-attenuation and 4=Self-interference from the original dataset. Then, perform kmeans clustering with the new dataset and considering PCA with 10 components and k in range(2, 10)

(code is already given below)

In [ ]:
labels_to_keep = [1,4]

#F: initialization (use the names below for the new dataset
X_truncated = X
y_truncated = y

for el in label_names:
    if el not in labels_to_keep:
        X_truncated = np.delete(X_truncated,np.where(y_truncated==el),0)
        y_truncated = np.delete(y_truncated,np.where(y_truncated==el),0)

print(X_truncated.shape)
print(y_truncated.shape)


numcomponents = 10
pca = PCA(numcomponents)
X_PCA = pca.fit_transform(X_truncated)

inertia = {}
silhouette = {}
for k in range(2, 10):
    km = cluster.KMeans(n_clusters=k, max_iter=1000).fit(X_PCA)
    inertia[k] = km.inertia_ # Inertia: Sum of distances of samples to their cluster center
    silhouette[k] = silhouette_score(X_PCA, km.labels_) 
plt.figure()
plt.plot(list(inertia.keys()), list(inertia.values()))
plt.xlabel("Number of clusters")
plt.ylabel("Inertia (with {} components)".format(numcomponents))
plt.show()

plt.figure()
plt.plot(list(silhouette.keys()), list(silhouette.values()))
plt.xlabel("Number of clusters")
plt.ylabel("Silhouette (with {} components)".format(numcomponents))
plt.show()



# Task 6b) 
Perform kmeans clustering of the truncated dataset of task 6a, considering k=2 and 10 PCA components. Then assign labels to data points and compare this partition with the ground truth (labels 1,4 in y_truncated) in terms of *rand_score*, *homogeneity_score* and *completeness_score*

Hints: 
- use *sklearn.metrics* to retrieve the above metrics. See documentation at 
https://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation 

(code is already given below)

In [ ]:
km = cluster.KMeans(n_clusters=2, max_iter=1000).fit(X_PCA)

y_pred = km.fit_predict(X_PCA)

rand = round(mt.rand_score(y_truncated, y_pred), 2)
homog = round(mt.homogeneity_score(y_truncated, y_pred), 2) # 1 if each cluster contains only members of a single class
complet = round(mt.completeness_score(y_truncated, y_pred), 2) # 1 if all members of a given class are assigned to the same cluster

print('Rand_score (0; 1): ' +str(rand))
print('Homogeneity (0; 1): ' +str(homog))
print('Completeness (0; 1): ' +str(complet))

